In [1]:
import librosa
import os
import numpy as np
import sys
from dtw import dtw
from numpy.linalg import norm
from numpy import array
import pyaudio
import wave

import heapq

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



**|Heap| = 80**  
**n_mfcc = 20**  
**DTWSeq = Window**  
**None**

In [2]:
def initialCorpus(path):
    # 音乐库位置
    audioList = os.listdir(path)

    raw_audioList = {}
    beat_database = {}

    for tmp in audioList:
        audioName = os.path.join(path, tmp)
        if audioName.endswith('.wav'):
            # 读入一维音频序列
            y, sr = librosa.load(audioName)
            # 提取 MFCC 特征
            f = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
            # 存入数据表
            beat_database[audioName] = f

    # 保存音乐节奏数据库
    np.save('beatDatabase_mfcc_20.npy', beat_database)
    
    return beat_database

In [3]:
def readCorpus(path):
    
    # 读入音乐节奏数据库
    all_data = np.load(path, allow_pickle=True)
    beat_database = all_data.item()
    
    return beat_database

In [4]:
def updateCorpus(path, dbpath):
    
    # 音乐库位置
    audioList = os.listdir(path)
    
    # 已保存序列的文件
    raw_db = readCorpus(dbPath)
    raw_files = raw_db.keys()
    
    for tmp in audioList:
        audioName = os.path.join(path, tmp)
        if audioName.endswith('.wav') and audioName not in raw_files:
            y, sr = librosa.load(audioName)
            # 提取 MFCC 特征
            f = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
            # 存入数据表
            beat_database[audioName] = f

    # 保存音乐节奏数据库
    np.save(dbpath, beat_database)

In [5]:
def voiceCompare_quick(dbPath, tPath):
    
    # 读入语料库
    all_data = np.load(dbPath, allow_pickle=True)
    beat_database = all_data.item()

    # 读入要识别的录音
    y, sr = librosa.load(tPath)

    # 识别录音的节奏序列
    tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
    beat_frames = librosa.feature.delta(beat_frames,mode ='nearest')
    x = array(beat_frames).reshape(-1, 1)

    # 将待识别的录音序列与语料库中语音逐一做DTW对比
    compare_result = {}
    
    for songID in beat_database.keys():
        y = beat_database[songID]
        y = array(y).reshape(-1, 1)
        
        dist = dtw(x, y).distance
        # print('两段话的差异程度为： ', songID.split("\\")[1], ": ", dist)
        
        compare_result[songID] = dist

    matched_song = min(compare_result, key=compare_result.get)
    print("最接近的录音是：", matched_song)

In [6]:
def normlize(data):
    n_mean = np.mean(data, axis=0)
    n_std  = np.std(data, axis=0)
    
    norm_data = np.divide(np.subtract(data, n_mean), n_std)
    return norm_data

In [7]:
from sklearn import preprocessing

def voiceCompare(dbPath, tPath):
    # ==== 最大检索数 ====
    aimNum = 80
    
    # 读入语料库
    all_data = np.load(dbPath, allow_pickle=True)
    beat_database = all_data.item()

    # ==== 读入要识别的录音 ====
    y, sr = librosa.load(tPath)

    # 提取录音的 MFCC 特征
    # x = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20).T  # n1 * 20
    x = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)  # 20 * n1
    lenx = len(x[0])
    
    # 标准化
#     x = x.T
#     for i in range(0, lenx):
#         x[i] = normlize(x[i])
#         # x[i] = preprocessing.minmax_scale(x[i])
#     x = x.T
    

    # ==== 将待识别的录音序列与语料库中语音逐一做DTW对比 ====
    
    # heap for [dist, 时间段，文件名]
    heap = []
    heapq.heapify(heap)  
    
    for songID in beat_database.keys():
        # 取出文件名对应的 mfcc 序列
        # y = beat_database[songID].T
        y = beat_database[songID]
        
        leny = len(y[0]) # 20 * n2 
        print(leny)
        
        # 标准化
#         y = y.T
#         for i in range(0, leny):
#             y[i] = normlize(y[i])
#             # y[i] = preprocessing.minmax_scale(y[i])
#         y = y.T

        for tp in range(0, leny - lenx):
            # *加速* 设定距离上限
            full = False  # 堆是否已满
            dist_UB = -10000  # DTW 距离上限
            overBound = False  # 是否过限
            
            if (len(heap) >= aimNum):
                full = True
                dist_UB = -heap[0][0]  # heap top (biggest) DTW dist as UB  
                
            # 计算 DTW(y[tp : tp + lenx])
            total_dist = 0
            
            for i in range(0, 20):
                # DTW dist
                total_dist += dtw(x[i], y[i][tp : tp + lenx], distance_only=False).distance
                
                # *加速* 超过上限直接取消
                if (full and total_dist > dist_UB):
                    overBound = True
                    break
            
            # *加速* 超过上限
            if (overBound):
                continue
            
            # 入栈
            tupleY = (-total_dist, tp, songID) # dtw 距离加负数转为大根堆
            
            heapq.heappush(heap, tupleY)
            if (len(heap) > aimNum):
                heapq.heappop(heap)
            
            print(tupleY)
            
        # end for
        
        
    return heap

In [8]:
def getTimePoint_dense(dbPath, tPath, vheap):
    res_num = 20 # 定义取出前 res_num 位的结果作为识别结果
    
    # 读入语料库
    all_data = np.load(dbPath, allow_pickle=True)
    beat_database = all_data.item()
    
    # 得到要识别的录音时长
    tTime = librosa.get_duration(filename=tPath)
    
    # 提取前 res_num 个相似的片段并输出对应时间段
    similar_n = heapq.nlargest(res_num, vheap)
    
    print("开始输出相似片段：")
    
    for i in range(0, res_num):
        music_name = similar_n[i][2]  # 录音文件名
        music_time = librosa.get_duration(filename=music_name)  # 录音时长
        
        music_pos = similar_n[i][1]  # 时间段所在帧数
        music_all = len(beat_database[music_name][0])  # 录音总帧数

        frag_st = music_time / music_all * music_pos  # 时间段起点
        frag_en = frag_st + tTime  # 时间段终点
        
        # print(music_name, music_time, music_pos, music_all, frag_st)
        # print("相似度第", i + 1, "位的为文件 ", music_name, "的 ", '%.2f' % frag_st, "到", '%.2f' % frag_en, "秒")
        
        print(music_name, ",", '%.2f' % frag_st, "秒,", '%.2f' % frag_en, "秒")


In [9]:
def getTimePoint(dbPath, tPath, vheap):
    # 读入语料库
    all_data = np.load(dbPath, allow_pickle=True)
    beat_database = all_data.item()
    
    # 得到要识别的录音时长
    tTime = librosa.get_duration(filename=tPath)
    
    heapq.nlargest(20, vheap)
    
    # ====== 对 vheap 进行去重 ======
    # 取出文件名
    name_set = set()
    for tp in vheap:
        name_set.add(tp[2])
    # print(name_set)
    
    # 合并下标差小于5的片段
    sheap = []
    for name in name_set:
        # 按下标排序
        nList = [x for x in vheap if x[2] == name]
        sortL = sorted(nList, key=lambda t:t[1])
        
        # 去重
        for tp in sortL:
            if len(sheap) < 1 or sheap[-1][2] != name or abs(sheap[-1][1] - tp[1]) > 5:
                sheap.append(tp)
            else:  
                if (sheap[-1][0] < tp[0]): 
                    sheap[-1] = tp  # 保留距离较小项

    # print(sheap)
    # 提取相似片段并输出对应时间段
    similar_n = sheap
    
    print("开始输出相似片段：")
    
    for i in range(0, len(sheap)):
        music_name = similar_n[i][2]  # 录音文件名
        music_time = librosa.get_duration(filename=music_name)  # 录音时长
        
        music_pos = similar_n[i][1]  # 时间段所在帧数
        music_all = len(beat_database[music_name][0])  # 录音总帧数

        frag_st = music_time / music_all * music_pos  # 时间段起点
        frag_en = frag_st + tTime  # 时间段终点
        
        # print(music_name, music_time, music_pos, music_all, frag_st)
        # print("相似度第", i + 1, "位的为文件 ", music_name, "的 ", '%.2f' % frag_st, "到", '%.2f' % frag_en, "秒")
        
        print(music_name, ",", '%.2f' % frag_st, "秒,", '%.2f' % frag_en, "秒")


In [10]:
# 语料库路径
corpus_path = './corpus'

# 数据表路径
dbPath = './beatDatabase_mfcc_20.npy';

# test file path
# testPath = './input/00415250-前5s.wav'
# testPath = './input/00429126-53s_60s.wav'
testPath = './input/00430105-hou5s.wav'

In [12]:
# 1 初始化语料序列库
# beatDB = initialCorpus(corpus_path)

# 2 更新语料库中新音乐文件的序列
# updateCorpus(corpus_path, dbPath)

# 3 读入语料序列库
# beat_database = readCorpus(dbPath)

vheap = voiceCompare(dbPath, testPath)

3876
(-351.2391074281186, 0, './corpus\\00415250.wav')
(-354.241866979748, 1, './corpus\\00415250.wav')
(-355.33284173347056, 2, './corpus\\00415250.wav')
(-355.28274957835674, 3, './corpus\\00415250.wav')
(-355.8065559770912, 4, './corpus\\00415250.wav')
(-356.57440343126655, 5, './corpus\\00415250.wav')
(-350.00678713805974, 6, './corpus\\00415250.wav')
(-342.1053788885474, 7, './corpus\\00415250.wav')
(-338.1353476922959, 8, './corpus\\00415250.wav')
(-335.579254116863, 9, './corpus\\00415250.wav')
(-328.49104747362435, 10, './corpus\\00415250.wav')
(-326.49133840948343, 11, './corpus\\00415250.wav')
(-323.6888167094439, 12, './corpus\\00415250.wav')
(-324.4110419936478, 13, './corpus\\00415250.wav')
(-324.951768392697, 14, './corpus\\00415250.wav')
(-322.77808464691043, 15, './corpus\\00415250.wav')
(-314.8334596324712, 16, './corpus\\00415250.wav')
(-304.82096169143915, 17, './corpus\\00415250.wav')
(-297.67019893042743, 18, './corpus\\00415250.wav')
(-293.7728428132832, 19, './co

(-305.27794799767435, 213, './corpus\\00415250.wav')
(-304.06184176169336, 214, './corpus\\00415250.wav')
(-305.6444218661636, 215, './corpus\\00415250.wav')
(-307.8479300495237, 216, './corpus\\00415250.wav')
(-308.53026818297803, 217, './corpus\\00415250.wav')
(-308.8650393988937, 218, './corpus\\00415250.wav')
(-308.8496182169765, 219, './corpus\\00415250.wav')
(-307.06942973844707, 220, './corpus\\00415250.wav')
(-304.84741797298193, 221, './corpus\\00415250.wav')
(-304.06748496741056, 222, './corpus\\00415250.wav')
(-302.52963934093714, 223, './corpus\\00415250.wav')
(-303.03143755346537, 224, './corpus\\00415250.wav')
(-301.5795027539134, 225, './corpus\\00415250.wav')
(-302.1100999042392, 226, './corpus\\00415250.wav')
(-306.07893341593444, 227, './corpus\\00415250.wav')
(-307.1247166134417, 228, './corpus\\00415250.wav')
(-307.59778335131705, 229, './corpus\\00415250.wav')
(-314.9420391470194, 230, './corpus\\00415250.wav')
(-317.2502732332796, 231, './corpus\\00415250.wav')
(-

(-299.1336777880788, 3000, './corpus\\00415250.wav')
(-293.332396697253, 3259, './corpus\\00415250.wav')
(-295.71587556786835, 3260, './corpus\\00415250.wav')
(-297.89704125374556, 3261, './corpus\\00415250.wav')
(-299.92931626550853, 3262, './corpus\\00415250.wav')
(-298.3429523129016, 3330, './corpus\\00415250.wav')
(-296.516284706071, 3331, './corpus\\00415250.wav')
(-298.07937692292035, 3332, './corpus\\00415250.wav')
(-298.4210039842874, 3333, './corpus\\00415250.wav')
(-293.3862788248807, 3342, './corpus\\00415250.wav')
(-294.0129189956933, 3343, './corpus\\00415250.wav')
(-297.5564089883119, 3344, './corpus\\00415250.wav')
(-296.7657461948693, 3370, './corpus\\00415250.wav')
(-296.37389192916453, 3371, './corpus\\00415250.wav')
(-293.11506675183773, 3372, './corpus\\00415250.wav')
(-293.8249286022037, 3373, './corpus\\00415250.wav')
(-294.7839143536985, 3374, './corpus\\00415250.wav')
(-295.25765080191195, 3375, './corpus\\00415250.wav')
(-296.31021694839, 3376, './corpus\\00415

In [13]:
getTimePoint(dbPath, testPath, vheap)

开始输出相似片段：
./corpus\00429239.wav , 8.29 秒, 10.36 秒
./corpus\00429239.wav , 72.33 秒, 74.40 秒
./corpus\00430105.wav , 20.27 秒, 22.34 秒
./corpus\00430105.wav , 33.11 秒, 35.18 秒
./corpus\00430105.wav , 40.40 秒, 42.47 秒
./corpus\00430105.wav , 73.69 秒, 75.76 秒
./corpus\00430105.wav , 76.01 秒, 78.08 秒
./corpus\00430105.wav , 131.69 秒, 133.75 秒
./corpus\00430105.wav , 144.62 秒, 146.69 秒
./corpus\00430105.wav , 144.83 秒, 146.90 秒
./corpus\00430105.wav , 146.85 秒, 148.92 秒
./corpus\00415250.wav , 4.69 秒, 6.76 秒
./corpus\00415250.wav , 13.35 秒, 15.42 秒
./corpus\00415250.wav , 27.49 秒, 29.56 秒
./corpus\00415250.wav , 49.99 秒, 52.06 秒
./corpus\00429126.wav , 11.31 秒, 13.38 秒
./corpus\00429126.wav , 41.91 秒, 43.98 秒
./corpus\00429126.wav , 96.15 秒, 98.22 秒
./corpus\00429126.wav , 164.85 秒, 166.92 秒
./corpus\00429881.wav , 180.00 秒, 182.06 秒
./corpus\00429881.wav , 185.27 秒, 187.34 秒


In [ ]:
inPara = sys.argv

if (len(inPara) < 2):
    print("请输入待识别录音文件路径！")
else:
    if (len(inPara) > 2):
        print("给定语料库路径为:", sys.argv[2])
        corpus_path = sys.argv[2]
    else:
        print("默认语料库路径为：", corpus_path)
    
    if (len(inPara) > 3):
        print("给定数据表路径为:", sys.argv[3])
        dbPath = sys.argv[3]
    else:
        print("默认数据表路径为：", dbPath)
    
    testPath = sys.argv[1]
    vheap = voiceCompare(dbPath, testPath)
    getTimePoint(dbPath, testPath, vheap)
  

In [11]:
testPath2 = './input/00430105-hou5s.wav'
vheap2 = voiceCompare(dbPath, testPath2)
getTimePoint(dbPath, testPath, vheap2)

3876
(-26229.914907246828, 0, './corpus\\00415250.wav')
(-26191.014937341213, 1, './corpus\\00415250.wav')
(-26435.58950202167, 2, './corpus\\00415250.wav')
(-26305.432218030095, 3, './corpus\\00415250.wav')
(-26515.76418789476, 4, './corpus\\00415250.wav')
(-26839.645168058574, 5, './corpus\\00415250.wav')
(-26805.525408051908, 6, './corpus\\00415250.wav')
(-26808.36786251515, 7, './corpus\\00415250.wav')
(-27200.364706151187, 8, './corpus\\00415250.wav')
(-26897.11939033121, 9, './corpus\\00415250.wav')
(-27266.729901842773, 10, './corpus\\00415250.wav')
(-27851.09544505924, 11, './corpus\\00415250.wav')
(-28119.597325734794, 12, './corpus\\00415250.wav')
(-28163.042064003646, 13, './corpus\\00415250.wav')
(-28203.675612293184, 14, './corpus\\00415250.wav')
(-26815.0607605204, 15, './corpus\\00415250.wav')
(-26637.597754709423, 16, './corpus\\00415250.wav')
(-26012.31332565844, 17, './corpus\\00415250.wav')
(-25228.324505493045, 18, './corpus\\00415250.wav')
(-25183.476823017, 19, '.

(-29506.412649113685, 718, './corpus\\00415250.wav')
(-29555.3908884041, 742, './corpus\\00415250.wav')
(-29136.234983008355, 743, './corpus\\00415250.wav')
(-28848.643764626235, 744, './corpus\\00415250.wav')
(-28920.069584991783, 745, './corpus\\00415250.wav')
(-29234.81632226333, 746, './corpus\\00415250.wav')
(-29012.61515871808, 747, './corpus\\00415250.wav')
(-28753.609211992472, 748, './corpus\\00415250.wav')
(-29150.982016664, 749, './corpus\\00415250.wav')
(-28512.39864415303, 758, './corpus\\00415250.wav')
(-28529.926762793213, 759, './corpus\\00415250.wav')
(-28960.039531935006, 760, './corpus\\00415250.wav')
(-28887.25089167431, 906, './corpus\\00415250.wav')
(-28612.977168682963, 907, './corpus\\00415250.wav')
(-28987.647181961685, 908, './corpus\\00415250.wav')
(-27830.757716307417, 1078, './corpus\\00415250.wav')
(-27126.142035344616, 1079, './corpus\\00415250.wav')
(-25387.408281916752, 1080, './corpus\\00415250.wav')
(-25551.264211112633, 1081, './corpus\\00415250.wav'

(-24761.30254136771, 4146, './corpus\\00429126.wav')
(-24741.699665032327, 4147, './corpus\\00429126.wav')
(-25313.2307016775, 4148, './corpus\\00429126.wav')
(-25795.512682855362, 4684, './corpus\\00429126.wav')
(-25497.158013224835, 4685, './corpus\\00429126.wav')
(-25644.136843089014, 6849, './corpus\\00429126.wav')
(-25614.07519461587, 6855, './corpus\\00429126.wav')
(-25808.565238583833, 6856, './corpus\\00429126.wav')
(-25356.23390646279, 7881, './corpus\\00429126.wav')
(-25549.83095984161, 7882, './corpus\\00429126.wav')
12231
(-25139.42802051641, 4300, './corpus\\00429239.wav')
(-24173.986298153177, 4301, './corpus\\00429239.wav')
(-23230.08639352955, 4302, './corpus\\00429239.wav')
(-22843.095606649294, 4303, './corpus\\00429239.wav')
(-22918.408371293917, 4304, './corpus\\00429239.wav')
(-23225.759904140607, 4305, './corpus\\00429239.wav')
(-23598.74580037035, 4306, './corpus\\00429239.wav')
(-23965.221420759335, 4307, './corpus\\00429239.wav')
(-24327.895678156987, 4308, './

In [12]:
getTimePoint(dbPath, testPath, vheap2)

开始输出相似片段：
./corpus\00430105.wav , 50.91 秒, 52.98 秒
./corpus\00429126.wav , 51.96 秒, 54.03 秒
./corpus\00429126.wav , 79.41 秒, 81.48 秒
./corpus\00429126.wav , 79.55 秒, 81.62 秒
./corpus\00429126.wav , 93.66 秒, 95.73 秒
./corpus\00429126.wav , 93.80 秒, 95.87 秒
./corpus\00429126.wav , 96.15 秒, 98.22 秒
./corpus\00429126.wav , 96.29 秒, 98.36 秒
./corpus\00429126.wav , 182.99 秒, 185.06 秒
./corpus\00415250.wav , 0.46 秒, 2.53 秒
./corpus\00415250.wav , 25.15 秒, 27.22 秒
./corpus\00415250.wav , 25.31 秒, 27.38 秒
./corpus\00415250.wav , 42.96 秒, 45.03 秒
./corpus\00415250.wav , 75.67 秒, 77.74 秒
./corpus\00429239.wav , 99.91 秒, 101.98 秒
./corpus\00429239.wav , 100.05 秒, 102.12 秒
./corpus\00429239.wav , 123.76 秒, 125.83 秒
./corpus\00429239.wav , 206.35 秒, 208.42 秒
